In [18]:
import pandas as pd
import numpy as np
import pymysql  
#import json
import csv
import os
#import re  
#import zipfile
from datetime import datetime, date, time, timedelta
from sqlalchemy import create_engine
from functools import reduce

import smtplib
from threading import Timer
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders
import math


###########################################
##### check if request have been done #####
###########################################
sql_con = pymysql.connect(host='10.45.0.6',
                                user='datarous',
                                password='X53N7rgQBBZdbXQaGCfpL7',
                                db='bietudes')



#---------SEND ACTUAL MAIL-----------------------------------------------------------------
def py_mail(msg, TO, FROM):  
    server = smtplib.SMTP('64.151.69.92:25')
    server.ehlo("adaware.com")    
    server.mail(FROM)
    server.rcpt(TO)
    
    if isinstance(msg, str):
        server.data(msg)
    else:
        server.data(msg.as_string())
    server.quit()
    
#---------SEND MAILWITH MULTIPLE TO AND CC-----------------------------------------------------------------
def send_email(message_text,message_subject, toaddr,cc, fromaddr):  
    server = smtplib.SMTP('64.151.69.92:25')
    server.ehlo("adaware.com")
    toaddrs = toaddr + cc
    if isinstance(message_text, str):
        print("msg is string")
        message = "From: %s\r\n" % fromaddr + "To: %s\r\n" % toaddr + "CC: %s\r\n" % ",".join(cc)+ "Subject: %s\r\n" % message_subject  + "\r\n"  + message_text
        server.sendmail(fromaddr, toaddrs, message)
    else: 
        print("msg is html table")
        server.sendmail(fromaddr, toaddrs, message_text.as_string())       
          
    server.quit()
        
# ----------------------CHECK FOR NULL METRICS FOR LAST DATE----------------------------------------------------
def check_nullmetrics(dfTocheck):
    df_firstrow=dfTocheck.iloc[0]
    nullColumns=''
    
    for col,row in df_firstrow.iteritems():         
        if col=='search_Bing':
            continue;    
        if df_firstrow[col]== 0 or pd.isnull(df_firstrow[col]):
            if nullColumns !='':
                 nullColumns=nullColumns+col+","
            else:
                 nullColumns=nullColumns+col             
        else:
            print("not 0")
        
        
    if nullColumns !='':
        print("Null columns are "+nullColumns )
        TO = ['shaina.puttur@adaware.com','bi@lavasoft.net']
        CC = ['shaina.puttur@adaware.com','bi@lavasoft.net']
        FROM = 'bi@lavasoft.net'
        subject = "Null Metrics"
        text = "Hi guys, \n"+"We have null metrics in "+ nullColumns+" from yesterday." +"\n\nFaranoosh is already looking into this"+"\n\nRegards,"+"\nShaina"
        send_email(text,subject,TO,CC, FROM)   
 
      
#--------COMPARES TWO LISTS AND RETURNS THE DIFFERENCE -------------------------------------------------------
def xor(list1, list2):
    outputlist = []
    list3 = list1 + list2
    for i in range(0, len(list3)):
        if ((list3[i] not in list1) or (list3[i] not in list2)) and (list3[i] not in outputlist):
             outputlist[len(outputlist):] = [list3[i]]
    return outputlist

#------------------CHECKING FOR MISSING HOURS(00-23) IN FACT_PLATFORM_AGG_COPY----------------------------------------------------------------
def missing_hours():
    mysql_q = (""" SELECT distinct createdate as Date FROM  bietudes.fact_platform_agg_copy 
                    where createdate>=curdate()-interval 1 day;
                """)

    df = pd.read_sql(mysql_q, con=sql_con)  
    df["Date"]=df["Date"].dt.strftime("%H")
    df["Date"]=df['Date'].astype(str).astype(int)
    allHoursList=list(range(0, 24))
    HourList = df['Date'].tolist()  
    diffList=xor(allHoursList,HourList)
    if diffList:
        print("Missing hours are")
        print(diffList)
        TO = ['shaina.puttur@adaware.com','bi@lavasoft.net']
        CC = ['shaina.puttur@adaware.com','bi@lavasoft.net']
        FROM = 'bi@lavasoft.net'
        subject = "H2O install missing hours"
        text = "Hi guys, \n"+"We are missing these hours: "+ ', '.join(str(x) for x in diffList)+" from yesterday.AWS didn't process last night." +"\n\nFaranoosh is already looking into this"+"\n\nRegards,"+"\nShaina"
        send_email(text,subject,TO,CC, FROM) 
    else:
        print("no missing hours")
        print(diffList)

    
#----------------------------------------------------------------------------------------------------------------------

missing_hours() 

mysql_q = ("""             
            SELECT  CreateDate,
            count(Type) as H2o_BundleInstallStart                  	
            FROM
            installer_data_fact
            WHERE 
            CreateDate > CURDATE() - INTERVAL 30 DAY and CURDATE()> CreateDate and type= 'BundleInstallStart'
            and isUnique=1
            GROUP BY 
            CreateDate order by CreateDate DESC
                """)

df = pd.read_sql(mysql_q, con=sql_con)    
df["CreateDate"] = df["CreateDate"].dt.strftime("%Y-%m-%d")
df.groupby(['CreateDate'], as_index=False).sum().sort_values(by=['CreateDate'], ascending=[False])
Installergroupby = df.groupby(['CreateDate'], as_index=False).sum().sort_values(by=['CreateDate'], ascending=[False]) 

mysql_q = ("""             
            SELECT   CreateDate,
            count(Type) as H2o_OfferShown                 	
            FROM
            installer_data_fact
            WHERE 
            CreateDate > CURDATE() - INTERVAL 30 DAY and CURDATE()> CreateDate and type= 'OfferShown'
            and isUnique=1
            GROUP BY 
            CreateDate order by CreateDate DESC
                """)

df = pd.read_sql(mysql_q, con=sql_con)    
df["CreateDate"] = df["CreateDate"].dt.strftime("%Y-%m-%d")
df.groupby(['CreateDate'], as_index=False).count().sort_values(by=['CreateDate'], ascending=[False])
OfferShowngroupby = df.groupby(['CreateDate'], as_index=False).sum().sort_values(by=['CreateDate'], ascending=[False]) 

mysql_q = ("""SELECT  CreateDate,sum(corrected_publisher_rev) as H2o_Publisher_Payout             
            FROM installer_data_fact where corrected_publisher_rev is not null 
            and CreateDate > CURDATE() - INTERVAL 30 DAY and CURDATE()> CreateDate
            and isUnique =1 group by CreateDate
                """)

df = pd.read_sql(mysql_q, con=sql_con)
df["CreateDate"] = df["CreateDate"].dt.strftime("%Y-%m-%d")
Publishergroupby = df.groupby(['CreateDate'], as_index=False).sum().sort_values(by=['CreateDate'], ascending=[False])

mysql_q = ("""SELECT  CreateDate,sum(corrected_adv_rev) as H2o_AdvertiserRevenue          
            FROM installer_data_fact where corrected_adv_rev is not null and 
            CreateDate > CURDATE() - INTERVAL 30 DAY and CURDATE()> CreateDate
            and isUnique =1 group by CreateDate
                """)
df= pd.read_sql(mysql_q, con=sql_con)
df["CreateDate"] = df["CreateDate"].dt.strftime("%Y-%m-%d")
Advertisersgroupby = df.groupby(['CreateDate'], as_index=False).sum().sort_values(by=['CreateDate'], ascending=[False])

mysql_q = ("""SELECT distinct inst_date as CreateDate,
              SUM(nb_installs) as WC_Installs
              FROM 
              r_data_installs  where inst_date > CURDATE() - INTERVAL 31 DAY and CURDATE()> inst_date
              group by inst_date ORDER BY 
              inst_date  DESC
                    """)

TotalInstalls = pd.read_sql(mysql_q, con=sql_con)
TotalInstalls['CreateDate'] = pd.to_datetime(TotalInstalls['CreateDate'])
TotalInstalls["CreateDate"]=TotalInstalls["CreateDate"].dt.strftime("%Y-%m-%d")


mysql_q = ("""
            SELECT
            dateId as CreateDate,
            SUM(CASE WHEN rsource = 'Bing' THEN revenue END) as search_Bing,
            SUM(CASE WHEN rsource = 'Yahoo' THEN revenue END) as search_Yahoo,
            SUM(CASE WHEN rsource = 'Yahoo_API' THEN revenue END) as search_Yahoo_API	
            FROM
            r_data_search_rev
            WHERE 
            dateId > CURDATE() - INTERVAL 31 DAY and CURDATE()> dateId
            GROUP BY 
            dateId order by dateId DESC
                     """)

RevenueGroupBy = pd.read_sql(mysql_q, con=sql_con)
RevenueGroupBy['CreateDate'] = pd.to_datetime(RevenueGroupBy['CreateDate'])
RevenueGroupBy["CreateDate"]=RevenueGroupBy["CreateDate"].dt.strftime("%Y-%m-%d")


mysql_q = ("""SELECT inst_date as CreateDate, sum( nb_installs*rate_per_inst)  as WC_Payout
            FROM r_data_installs  WHERE 
            inst_date > CURDATE() - INTERVAL 31 DAY and CURDATE()> inst_date 
            group by inst_date 
            order by inst_date desc
                     """)

PayoutGroupBy = pd.read_sql(mysql_q, con=sql_con)
PayoutGroupBy['CreateDate'] = pd.to_datetime(PayoutGroupBy['CreateDate'])
PayoutGroupBy["CreateDate"]=PayoutGroupBy["CreateDate"].dt.strftime("%Y-%m-%d")


dfs = [Installergroupby,OfferShowngroupby,Publishergroupby,Advertisersgroupby,TotalInstalls,PayoutGroupBy,RevenueGroupBy]
df_final = reduce(lambda left,right: pd.merge(left,right,on='CreateDate', how='outer'), dfs)
# dfnew=df_final.iloc[0]
# dfnew.loc['H2o_OfferShown'] = 0
# dfnew.loc['H2o_BundleInstallStart'] = 0
# headlamp = pd.DataFrame(dfnew)

check_nullmetrics(df_final)
# df_final.to_csv('C:/Users/shaina.puttur/Documents/db/DailyReport.csv',index=False)

# ----------Formatting dataframe values such as currency and large nos---------------------------------

format_mapping={'H2o_BundleInstallStart': '{:,.0f}','H2o_Publisher_Payout': '${:,.2f}', 'WC_Payout': '${:,.2f}',
                'search_Bing': '${:,.2f}','search_Yahoo': '${:,.2f}', 'search_Yahoo_API': '${:,.2f}',
                'H2o_OfferShown': '{:,.0f}','H2o_AdvertiserRevenue': '${:,.2f}', 
                'WC_Installs': '{:,.0f}'
               }

for key, value in format_mapping.items():
     df_final[key] = df_final[key].apply(value.format)
        
# # -------------------EMAIL WITH CSV AS EMAIL BODY-------------------------------------------------------   

dir_path = os.path.dirname(os.path.realpath('__file__'))
df_final.to_html(dir_path+'\\DailyReport.html',index=False)

# FROM_EMAIL = 'bi@lavasoft.net'
# # TO_EMAIL = 'bandith.nasavan@adaware.com'
# TO_EMAIL = 'shaina.puttur@adaware.com'
# SUBJECT = 'Daily Report'

# msg = MIMEMultipart()
# msg['From'] = FROM_EMAIL
# msg['To'] = COMMASPACE.join([TO_EMAIL])
# msg['Subject'] =SUBJECT

# f = open(dir_path+'\\DailyReport.html')
# attachment = MIMEText(f.read(),'html')
# msg.attach(attachment)
# body=f.read()
# f.close()
# py_mail(msg, TO_EMAIL, FROM_EMAIL)

msg = MIMEMultipart('alternative')
TO = ['shaina.puttur@adaware.com','bi@lavasoft.net']
CC = ['shaina.puttur@adaware.com','bi@lavasoft.net']
FROM = 'bi@lavasoft.net'
subject = "daily report"
msg = MIMEMultipart()
msg['From'] = FROM
msg['To'] =", ".join(TO)
msg['Cc'] =", ".join(CC)
msg['Subject'] =SUBJECT

f = open(dir_path+'\\DailyReport.html')
attachment = MIMEText(f.read(),'html')
msg.attach(attachment)
f.close()
send_email(msg,subject, TO,CC, FROM)

# ------------------CSV AS ATTACHMENT---------------------------------------------------
# part = MIMEBase('application', "octet-stream")
# part.set_payload(open(FILEPATH, "rb").read())
# encoders.encode_base64(part)
# part.add_header('Content-Disposition', 'attachment', filename=FILENAME)  # or
# # part.add_header('Content-Disposition', 'attachment; filename="attachthisfile.csv"')
# msg.attach(part)

#     msg = msg.format(fromaddr =fromaddr, toaddr = toaddrs[0])
#--------------------------------------------------------------------------------------------


sql_con.close()




no missing hours
[]
not 0
not 0
not 0
not 0
not 0
not 0
not 0
not 0
not 0
msg is html table
